In many ML problems we can't have the exact anwser, and we must use deterministic approximate algorithms, or Monte Carlo approximations.

# Sampling and Monte Carlo methods

We ca draw samples from a probability distribution and compute a MC estimate of some desired quantity.  
We can use it to provide a speedup for a costly but tractable sum (eg SGD), or for an untratable sum (eg gradient of the log partition function).  
Or we can train a model to sample from the data distribution.

## Basics of Monte-Carlo sampling

The idea is to view the sum as an expectation under some distribution $p$:
$$s = \sum_{x} p(f)f(x) = \mathbb{E}_p[f(x)]$$

We can approximate $s$ by drawing $n$ samples $x^{(i)}$ from $p$ and taking the average:
$$\hat{s}_n = \frac{1}{n} \sum_{i=1}^n f(x^{(i)})$$  

$\hat{s}_n$ is an unbiased estimator that converges to $s$ when $n \to \infty$, prodived that $\text{Var}[f(x^{(i)})]$ is bounded.  

$$\text{Var}(\hat{s}_n) = \frac{\text{Var}(f(x))}{n}$$

We can compute an empircal average of $f(x)$ and their empircal variance, and get an estimate of the variance of $s$. With these we can estimate confidence intervals around the estimate $\hat{s}_n$.  

But all this is only possible when we can easily sample from $p$. Otherwhise we must use technique such as importance sampling or Monte Carlo Markov Chains.

## Importance Sampling

When estimating $s$:
$$s = \sum_{x} p(f)f(x) = \mathbb{E}_p[f(x)]$$

we need to device which part is the distribution $p$, and which part is the quantity $f$.  
We can rewrite it as:
$$p(x)f(x) = q(x) \frac{p(x)f(x)}{q(x)}$$

And now we can sample from $q$:
$$s = \mathbb{E}_q \left[ \frac{p(x)}{q(x)}f(x) \right]$$

Sometimes we need to compute an expectation for a given $p$ and $f$, however $p$ might not be the best choice to get enough samples. We can derive $q^*$, what is called optimal importance sampling. 

The MC estimate:
$$\hat{s}_p = \frac{1}{n} \sum_{i=1, x^{(i)} \sim p}^n f(x^{(i)})$$  

Can be transformed to an importance sampling estimator:
$$\hat{s}_q = \frac{1}{n} \sum_{i=1, x^{(i)} \sim q}^n \frac{p(x^{(i)})}{q(x^{(i)})} f(x^{(i)})$$  

Both are unbiased estimates of $s$, but the variance depends greatly on the choice of $q$:

$$\text{Var}(\hat{s}_q) = \frac{\text{Var}(\frac{p(x)}{q(x)}f(x))}{n}$$

The minimum variance occurs when $q$ is:
$$q^*(x) = \frac{p(x)|f(x)|}{Z}$$
with $Z$ normalization constant so that $q^*(x)$ sums to $1$  

When $f(x)$ does not change sign, $\text{Var}(\hat{s}_{q^*}) = 0$ (a single sample is sufficient).  
But usually sampling from $q^*$ is unfeasible, and we must find a feasible $q$ tha reduces the variance.  

Another approach is biased importance sampling, they do not required normalized $p$ or $q$:
$$\hat{s}_\text{BIS} = \frac{\sum_{i=1}^n \frac{\tilde{p}(x^{(i)})}{\tilde{q}(x^{(i)})} f(x^{(i)})}{\sum_{i=1}^n \frac{\tilde{p}(x^{(i)})}{\tilde{q}(x^{(i)})}}$$

with $\tilde{p}$ and $\tilde{q}$ unormalized forms of $p$ and $q$.  

Good choice of $q$ greatly improve the MC estimate, but poor choices do the opposite.  
The $q$ distribution is usually a simple and easy to sample from distribution.  
We can get very large variance when $q(x)$ is tiny and neither $f(x)$ or $p(x)$ are small enough to cancel it. But this happens rarely, at least with low-dimensional $x$. With higher-dimensional $x$ this happens more often.  

Nevertheless, importance sampling is often used in DL: estimate a parition function, or the log-ikelihood in variational autoencoders.

# Markov Chain Monte Carlo Methods

Wheh we can't sample from $p$ (often because $p_\text{model}$ is an undirected model), we can use MCMC to get approximate samples.  
We get guarantees of good sampling when $p(x)$ is nonzero for every state, which is the case for Energy-based models for example.  

In a EBM with $p(a,b)$, in order to sample $a$ we need to sample from $p(a|b)$, for $b$ it's $p(b|a)$, which is an infinite problem.  

With a Markov Chain, we start with an intial state $x$. We a define a transition distribution $T(x'|x)$, and repeatedly update state $x$ to a new state $x' \sim T(x|x')$. Over time, $x$ becomes a faire sample from $p(x)$.  

Let suppose $x$ random variable with countably many states. If we run infintely many markov chains, all states are drawn from $q^{(t)}(x)$, with $t$ number of ellapsed time step. $q^{(0)}$ is some arbitraty distribution to initialize $x$, and we would like $q^{(t)}$ to converge to $p$.  

Because $x$ is a positive integer, the probability distribution $q$ can be described with a vector $v$:
$$q(x = i) = v_i$$

We can define $q^{(t+1)}$ given $q^{(t)}$ by using $T(x'|x)$:
$$q^{(t+1)}(x') = \sum_x q^{(t)} (x) T(x'|x)$$
T$(x'|x)$ can be represent by a matrix:
$$A_{ij} = T(x'=i|x=j)$$
We can now express the update equation in matrix form:
$$v^{(t)} = Av^{(t-1)}$$
$$v^{(t)} = A^tv^{(0)}$$

A is a special matrix called a stochastic matrix, and it can be shown than the largest eigenvalue of $A$ is 1. If we rewrite $A$ in terms of it's diagonalization, we get:
$$v^{(t)} = Vdiag(\Lambda)^t V^{-1}v^{(0)}$$

Over time, all eigenvalues $< 1$ decay to $0$, and $A$ is left with a single eigenvector with eigenvalue $1$. In this situation, we arrive to a stationary distribution:
$$v' = Av = v$$  


If we chose $T$ correctly, then the stationary distriution $q$ will be equal to $p$.  

Al those properties works for coutinous variables, but this is sometimes called a Harris chain:
$$q'(x') = E_{x \sim q} T(x'|x)$$


Running a markov chain until reaching equilibrium is called burning in the Markov chain. be way draw samples from this equilibrium, they are all identically distributed, but two successive samples will be highly correlated. We can mitigate this problem by returning only one every $n$ samples.  
MCMCM is expensive because of the time needed to burn in, and to produce all samples.  
To get trully independant samples, we can run multiples markov chains in parallel.  
In DL, we usually use several Markov Chains, and obtain several samples from each.  
The number of steps before burn-in is called the mixing time. We don't know how long, and it's difficult to test is a markov chain ha reached equilibrium.  
Instead, we simply run MCMCM for a fixed amount of time that we roughly estimate to be sufficient.

# Gibbs Sampling

In DL, we derive $T$ from $p_\text{model}(x)$, usually a EBM.  

In Gibbs sampling, sampling from $T(x'|x)$ is accomplished by selecting one $x_i$, and sampling it from $p_\text{model}$ conditionned on its neighbors. We can also sample several variables at once if they are independant given all their neighbors (In a RBM, we can sample from all $h$/$v$ simultaneously).  

Other approches, such as Metropolis-Hastings, are used in othen contexts than DL

# The Challenge of Mixing between Separated Modes

MCMCM, especially with high-dimensional data, mix poorly, samples become very correlated. This is called slow mixing.
The chain make small steps, with similar energy value between steps. It tends co convergence to a mode with low energy, and get stuck on that mode. Escape route are rare, the the markov chain will continue to sample the same mode longer than it should.  

Transition between two modes separated by a high energy barrier is exponentially unlikely.  
it's very expensive to obtain a reliable set of samples covering all modes because of the difficulty of mixing between modes.

In DL models, we often define a model $p_\text{model}(x,h)$, and draw samples of $x$ by alternating sampling from $p_\text{model}(x|h)$ and $p_\text{model}(h|x)$.  
To mix efficiently, we would like $p_\text{model}(h|x)$ to have very high entropy.  
To have a great representation $h$ of $x$, we would like $h$ and $x$ to have very high informations.  
Tese are 2 opposite goals. The shaper is the distribution learned by an RBM, the harder it is to sample from it.

## Tempering to Mix between Modes

We can fix slow mixing by construting alternate version of the model, with lower peaks and higher valleys.  
Energy-based models can be augmented with a parameter $\beta$ that controls how sharp the distribution is:
$$p_\beta(x) \propto \exp (-\beta E(x))$$

We can draw samples by using $\beta < 1$. These are markor chain based on tempered transitions. They temporaly sample with $\beta < 1$ to mix different modes, then return to $\beta = 1$. 
Another approach, paraller tempering, simulates many states at different $\beta$.  
Those techniques are used for RBMs and others, but they did not solve the mixing problem for complex models.

## Depth May Help Mixing

Using stacks of autoencoders or RBMs learn yields a $p(x,h)$ more spead out and uniform, with less gap between the different modes, thus allowing to mix faster between modes.